In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models, transforms
from src.dataset.coco import get_coco
import src.dataset.DatasetReader as DatasetReader
import matplotlib.pyplot as plt
import src.dataset.utils as utils
import time

In [3]:
print(torch.version.cuda)  # Check CUDA version PyTorch was built with
print(torch.cuda.is_available())  # Check if PyTorch recognizes CUDA
print(torch.backends.cudnn.enabled)  # Check if cuDNN is available

print(torch.cuda.is_available())  # Should return True
print(torch.cuda.get_device_name(0))  # Should show RTX 3060


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)


12.6
True
True
True
NVIDIA GeForce RTX 3080
cuda


In [4]:

transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),  
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  
])

dataset = DatasetReader.COCODataset(annotation_file='../data/annotations/annotations/instances_train2017.json',
    image_dir= '../data/train2017/train2017',
    target_classes=[s.lower() for s in utils.GLOBAL_CLASSES],
    transform=transform)

batch_size = 256
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=11, prefetch_factor=4, persistent_workers=True)

model = models.mobilenet_v3_small(pretrained=True)
model.classifier[3] = nn.Linear(model.classifier[3].in_features, 64)  
model = model.to(device)

print(len(dataset))
print(dataset[90001][1])

e:\projects\lpcv\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
e:\projects\lpcv\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


532797
31


In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.00002)

def train_model(model, dataloader, criterion, optimizer, num_epochs=50):
    model.train()
    for epoch in range(num_epochs):
        start_time = time.time()
        running_loss = 0.0
        for i, (images, labels) in enumerate(dataloader):

            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            if (i + 1) % 10 == 0:
                print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(dataloader)}], Loss: {loss.item():.4f}")
        asdf = time.time() - start_time
        print(f"Epoch [{epoch+1}/{num_epochs}], Average Loss: {running_loss / len(dataloader):.4f}, Time: {asdf:.4f}s")
        torch.save(model.state_dict(), f"model_{epoch}epoha_coco_train_batch_size_{batch_size}.pth")

In [ ]:
# run this to train the model 
# train_model(model, dataloader, criterion, optimizer)

: 

# Evaluation
- treba da se stavi da koristi validacioni dataaset

In [ ]:
model.eval()

totalPost = 0
accuracyPost = 0


for i in range(0, len(dataset)):
    totalPost += 1
    image, label = dataset[i]
    image = image.unsqueeze(0).to(device)
    label = torch.tensor([label]).to(device)
    output = model(image)
    _, predicted = torch.max(output, 1)
    # print(f"Predicted: {predicted.item()}, Actual: {label.item()}")
    if(i % 1000 == 0):
        print(f"Slika: {i}")
    if predicted.item() == label.item():
        accuracyPost += 1
    

print(f"Accuracy: {accuracyPost/totalPost}")

Slika: 0


In [ ]:
# ovo save-uje model
torch.save(model.state_dict(), "model_50epoch_coco_val_batch_size_384_acc_79.2.pth")

In [5]:
model2 = models.mobilenet_v3_small(pretrained=True)
model2.classifier[3] = nn.Linear(model2.classifier[3].in_features, 64)  
model2 = model2.to(device)


model2.train()

for param in model2.features.parameters():

    param.requires_grad = False



for param in model2.classifier.parameters():

    param.requires_grad = True




In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model2.parameters(), lr=0.00002)

train_model(model2, dataloader, criterion, optimizer, num_epochs=50)

Epoch [1/50], Step [10/61], Loss: 4.2363
Epoch [1/50], Step [20/61], Loss: 4.1546
Epoch [1/50], Step [30/61], Loss: 4.1426
Epoch [1/50], Step [40/61], Loss: 4.0789
Epoch [1/50], Step [50/61], Loss: 4.0633
Epoch [1/50], Step [60/61], Loss: 4.0121
Epoch [1/50], Average Loss: 4.1151, Time: 33.5041s
Epoch [2/50], Step [10/61], Loss: 3.9519
Epoch [2/50], Step [20/61], Loss: 3.9103
Epoch [2/50], Step [30/61], Loss: 3.9291
Epoch [2/50], Step [40/61], Loss: 3.8680
Epoch [2/50], Step [50/61], Loss: 3.7918
Epoch [2/50], Step [60/61], Loss: 3.8115
Epoch [2/50], Average Loss: 3.8920, Time: 11.0812s
Epoch [3/50], Step [10/61], Loss: 3.7200
Epoch [3/50], Step [20/61], Loss: 3.7200
Epoch [3/50], Step [30/61], Loss: 3.6952
Epoch [3/50], Step [40/61], Loss: 3.7067
Epoch [3/50], Step [50/61], Loss: 3.6115
Epoch [3/50], Step [60/61], Loss: 3.6578
Epoch [3/50], Average Loss: 3.7144, Time: 11.0041s
Epoch [4/50], Step [10/61], Loss: 3.6658
Epoch [4/50], Step [20/61], Loss: 3.5226
Epoch [4/50], Step [30/61],

: 